In [11]:
import sys
assert sys.version_info >= (3, 5)  # make sure we have Python 3.5+
from pyspark.sql import SparkSession, functions, types
from pyspark.conf import SparkConf

def read_input():
    ebird_schema = types.StructType([
        types.StructField("speciesCode",types.StringType()),
        types.StructField("comName",types.StringType()),
        types.StructField("sciName",types.StringType()),
        types.StructField("locId",types.StringType()),
        types.StructField("locName",types.StringType()),
        types.StructField("obsDt",types.StringType()),
        types.StructField("howMany",types.StringType()),
        types.StructField("lat",types.StringType()),
        types.StructField("lng",types.StringType()),
        types.StructField("obsValid",types.StringType()),
        types.StructField("obsReviewed",types.StringType()),
        types.StructField("locationPrivate",types.StringType()),
        types.StructField("subId",types.StringType()),
    ])
    df = spark.read.format("json").schema(ebird_schema).option("recursiveFileLookup", "true").option("multiLine","true").load("gs://big-data-1-project-storage/ebirdDataYearly")
    print((df.count(),len(df.columns)))
    #df.write.save("gs://big-data-1-project-storage/cleaned-data/ebird.csv",format='csv',header=True)

if __name__=='__main__':
    spark = SparkSession.builder.appName('ebird').getOrCreate()
    sc = spark.sparkContext
    read_input()

(1868263, 13)


In [8]:
df_unclean = spark.read.format("csv").option("header","true").csv("gs://big-data-1-project-storage/cleaned-data/ebird.csv")
df_clean = df_unclean.filter(df_unclean.howMany.isNotNull())
print(df_clean.show(5))

+-----------+--------------------+------------------+--------+--------------------+----------------+-------+----------+------------+--------+-----------+---------------+---------+
|speciesCode|             comName|           sciName|   locId|             locName|           obsDt|howMany|       lat|         lng|obsValid|obsReviewed|locationPrivate|    subId|
+-----------+--------------------+------------------+--------+--------------------+----------------+-------+----------+------------+--------+-----------+---------------+---------+
|     nswowl|Northern Saw-whet...| Aegolius acadicus| L787133|       Kumdis Slough|2017-03-20 23:15|      1|53.7392187|-132.1612358|    true|      false|           true|S35611913|
|    wilsni1|      Wilson's Snipe|Gallinago delicata|L1166559|Hornby Island--Fo...|2017-03-20 21:44|      1|49.4973435|-124.6768427|    true|      false|          false|S35323282|
|    cacgoo1|      Cackling Goose| Branta hutchinsii| L833055|Central Saanich--...|2017-03-20 19:24|

In [9]:
print((df_clean.count(),len(df_clean.columns)))

(1564571, 13)


In [10]:
df_clean.write.save("gs://big-data-1-project-storage/cleaned-data/ebird_nonull.csv",format='csv',header=True)